### yolov7 공식 레포지토리를 가져와서 scratch로 구현 테스트

In [12]:
# 모델 로드
from  hubconf import custom
import numpy as np
import cv2

model = custom(path_or_model='weights/240501_best.pt')  # custom example
# model = create(name='yolov7', pretrained=True, channels=3, classes=80, autoshape=True)  # pretrained example


                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

Adding autoShape... 


In [28]:
# 인퍼런스
# Verify inference
img = cv2.imread('test_image.png')
results = model(img).pandas().xyxy[0]  # batched inference
print(results)

         xmin        ymin        xmax        ymax  confidence  class    name
0  598.207397   66.344719  755.817139  406.846863    0.822754      4  person
1  102.165428  114.544327  340.649994  420.449219    0.799316      4  person
2  417.828491   71.963081  746.354675  417.492188    0.791992      4  person
3    0.036963  103.011909   85.014648  415.717957    0.722656      4  person
4  392.693726   77.655365  503.878113  284.130066    0.676270      4  person
5  250.904099   89.113861  464.253876  419.857819    0.675293      4  person
6    1.108887  243.766586  169.142181  420.449219    0.661621      4  person
7   40.954880   73.441597  145.929489  251.159164    0.657227      4  person
8  295.703125    1.290040  413.392944  109.073822    0.624512      4  person
9  171.507812    0.000000  293.041779  134.208588    0.287354      4  person


In [32]:
# 후처리
dic_list = []
for idx, row in results.iterrows():
    bbox = [int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])]
    conf = row['confidence']
    class_no = row['class']
    name = row['name']
    dic_list.append({'bbox':bbox, 'conf':conf, 'class_no':class_no, 'name':name})
print(dic_list[0])

{'bbox': [598.2073974609375, 66.34471893310547, 755.817138671875, 406.84686279296875], 'conf': 0.82275390625, 'class_no': 4, 'name': 'person'}


In [33]:
from hubconf import custom
import time
import cv2

# 모듈 제작 테스트
class custom_yolov7_run:
    '''
    240524
    YOLOv7 공식 레포지토리 참고하여 numpy 이미지를 입력하면 dic으로 이루어진 list형태로 인퍼런스 결과가 도출되도록 만든 모듈
    '''
    def __init__(self, model_path):
        '''
        model_path: YOLOv7 wieght파일 경로
        '''
        self.model = custom(path_or_model = model_path)
        
    def detect(self, bgr_img):
        '''
        이미지를 입력하면 dic으로 이루어진 list를 반환
        img: cv2로 로드한 bgr 이미지를 입력(함수 안에서 rgb로 변환함)
        '''
        # 이미지 bgr -> rgb
        self.bgr_img = bgr_img # 나중에 그릴 때 사용
        self.img = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2RGB)
        # 인퍼런스
        start = time.time()
        results = self.model(self.img).pandas().xyxy[0]
        spent_time = round(time.time() - start, 3)
        # 후처리
        self.dic_list = []
        for idx, row in results.iterrows():
            bbox = [int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])]
            conf = round(row['confidence'], 3)
            class_no = row['class']
            name = row['name']
            self.dic_list.append({'bbox':bbox, 'conf':conf, 'class_no':class_no, 'name':name, 'inf_time':spent_time})
        return self.dic_list
    
    def draw(self):
        '''
        self.img에 self.dic_list로 그림그려진 이미지를 반환
        '''
        for dic in self.dic_list:
            cv2.rectangle(self.bgr_img, (dic['bbox'][0], dic['bbox'][1]), (dic['bbox'][2], dic['bbox'][3]), (0,0,255), 2)
            text = f'{dic["name"]}:{dic["conf"]}'
            cv2.putText(self.bgr_img, text, (dic['bbox'][0], dic['bbox'][1]+25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        return self.bgr_img

In [34]:
# 240514 Real Sense 카메라 사용 모듈
import pyrealsense2 as rs
import numpy as np
import cv2

class real_sense:
    def __init__(self):
        '''
        Real Sense 카메라 연결하여 파이썬으로 RGB 이미지와 Depth Map을 불러오는 Class.
        공식 문서를 참고하여 사용하기 쉽게 Class의 형태로 제작함
        '''
        # Configure depth and color streams
        self.pipeline = rs.pipeline()
        config = rs.config()

        # Get device product line for setting a supporting resolution
        pipeline_wrapper = rs.pipeline_wrapper(self.pipeline)
        pipeline_profile = config.resolve(pipeline_wrapper)
        device = pipeline_profile.get_device()
        device_product_line = str(device.get_info(rs.camera_info.product_line))

        found_rgb = False
        for s in device.sensors:
            if s.get_info(rs.camera_info.name) == 'RGB Camera':
                found_rgb = True
                break
        if not found_rgb:
            print("The demo requires Depth camera with Color sensor")
            exit(0)

        config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

        # Start streaming
        self.pipeline.start(config)
        print('Real Sense 카메라 수신 설정 완료')

    def get_cam(self):
        '''
        RealSense 카메라에서 frame들을 가져옴.
        이 frame들에서 RGB이미지, Depth맵 등 다양한 이미지가 추출됨
        frame이 return되지는 않고, 내부 변수로 활용됨
        '''
        self.frames = self.pipeline.wait_for_frames()
    
    def get_color_img(self):
        '''
        get_cam 실행 이후에 실행되어야 에러가 발생하지 않음
        self.frames라는 변수가 존재해야 실행 가능한 로직이기 때문임
        '''
        color_frame = self.frames.get_color_frame()
        if color_frame:
            self.color_image = np.asanyarray(color_frame.get_data())
            return self.color_image
        else:
            print('get_color_img 에러 발생!')
        
    def get_depth_img(self):
        '''
        get_cam 실행 이후에 실행되어야 에러가 발생하지 않음
        self.frames라는 변수가 존재해야 실행 가능한 로직이기 때문임
        '''
        depth_frame = self.frames.get_depth_frame()
        if depth_frame:
            self.depth_image = np.asanyarray(depth_frame.get_data())
            return self.depth_image
        else:
            print('get_depth_img 에러 발생!')

    def get_depth_color_map(self):
        '''
        self.depth_image로부터 color_map을 만들어서 반환함
        반드시 get_depth_img 함수가 실행되어야 에러가 발생하지 않음
        self.depth_image가 존재해야 실행 가능한 로직이기 때문임
        '''
        self.depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(self.depth_image, alpha=0.03), cv2.COLORMAP_JET)
        return self.depth_colormap
    
    def concat_all(self):
        '''
        위에서 만든 color_img, depth_map, depth_color_map 3개를 가로 Concat하여 반환
        위 3개 이미지가 모두 존재해야 에러가 발생하지 않음
        위 3개 함수를 선언하지 않고 코드를 실행하면 이전 frame의 이미지가 concat될 것임(주의 필요)
        '''
        # 깊이 이미지의 최대값을 확인하고, 8비트 형식으로 스케일 조정
        scaled_depth_image = cv2.convertScaleAbs(self.depth_image, alpha=(255.0/np.max(self.depth_image)))
        depth_3d = cv2.cvtColor(scaled_depth_image, cv2.COLOR_GRAY2BGR) # 1차원인 depth 이미지를 3차원으로 변경함
        concatenated_image = cv2.hconcat([self.color_image, depth_3d, self.depth_colormap])
        return concatenated_image

In [35]:
# 리얼센스 카메라 스트리밍 테스트
RealSense = real_sense()
while True:
    RealSense.get_cam() # 카메라 수신
    color_img = RealSense.get_color_img()
    depth_img = RealSense.get_depth_img()
    depth_color_map = RealSense.get_depth_color_map()
    cv2.imshow('Real Sense Streaming[color_img, depth_img, depth_color_map]', RealSense.concat_all())
    cv2.imshow('depth map', RealSense.depth_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

Real Sense 카메라 수신 설정 완료


In [36]:
# Real Sense로 영상 받아서 YOLOv7 인퍼런스 테스트
RealSense = real_sense()
model = custom_yolov7_run(model_path = 'weights/240501_best.pt')
while True:
    RealSense.get_cam() # 카메라 수신
    color_img = RealSense.get_color_img()
    # depth_img = RealSense.get_depth_img()
    # depth_color_map = RealSense.get_depth_color_map()
    result = model.detect(bgr_img = color_img)
    print(result)
    cv2.imshow('YOLOv7 test', model.draw())
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break


                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

Real Sense 카메라 수신 설정 완료


105   [102, 103, 104]  1    129240  models.yolo.Detect                      [19, [[12, 16, 19, 36, 40, 28], [36, 75, 76, 55, 72, 146], [142, 110, 192, 243, 459, 401]], [256, 512, 1024]]
Model Summary: 407 layers, 37291640 parameters, 37291640 gradients, 105.4 GFLOPS

YOLOR  e762db2 torch 2.2.1 CUDA:0 (NVIDIA GeForce RTX 3060, 12287.5MB)



Adding autoShape... 
[{'bbox': [14, 67, 477, 479], 'conf': 0.763, 'class_no': 4, 'name': 'person', 'inf_time': 0.172}, {'bbox': [464, 164, 640, 339], 'conf': 0.754, 'class_no': 12, 'name': 'TV', 'inf_time': 0.172}, {'bbox': [0, 163, 112, 289], 'conf': 0.463, 'class_no': 7, 'name': 'chair', 'inf_time': 0.172}]
[{'bbox': [13, 70, 476, 480], 'conf': 0.756, 'class_no': 4, 'name': 'person', 'inf_time': 0.018}, {'bbox': [464, 163, 640, 340], 'conf': 0.756, 'class_no': 12, 'name': 'TV', 'inf_time': 0.018}, {'bbox': [0, 164, 114, 289], 'conf': 0.523, 'class_no': 7, 'name': 'chair', 'inf_time': 0.018}]
[{'bbox': [464, 164, 640, 339], 'conf': 0.757, 'class_no': 12, 'name': 'TV', 'inf_time': 0.015}, {'bbox': [7, 71, 475, 479], 'conf': 0.753, 'class_no': 4, 'name': 'person', 'inf_time': 0.015}, {'bbox': [0, 163, 114, 290], 'conf': 0.441, 'class_no': 7, 'name': 'chair', 'inf_time': 0.015}]
[{'bbox': [2, 71, 475, 479], 'conf': 0.773, 'class_no': 4, 'name': 'person', 'inf_time': 0.014}, {'bbox': [464